In [297]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn.model_selection as model_selection
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

import json
import pandas as pd 
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt 

import optuna 
from optuna.integration import XGBoostPruningCallback

gpd.io.file.fiona.drvsupport.supported_drivers["KML"] = "rw"

In [298]:
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

In [299]:
def objective(trial, train_x, train_y, test_x, test_y):
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dtest = xgb.DMatrix(test_x, label=test_y)

    param = {
        "silent": 1,
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "nrounds" : trial.suggest_int("nrounds", 100, 1000),
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 10)
        param["eta"] = trial.suggest_loguniform("eta", 1e-8, 2.0)
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
        param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    bst = xgb.train(param, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    
    preds = bst.predict(dtest)
    rmsle = root_mean_squared_log_error(np.exp(test_y), np.exp(preds))
    
    return rmsle

In [300]:
# Read the building datasets
buildings_train = pd.read_csv("resources/data/buildings_train.csv").set_index("id")
buildings_train["split"] = "train"
buildings_test = pd.read_csv("resources/data/buildings_test.csv").set_index("id")
buildings_test["split"] = "test"

# Create a GeoDataFrame of all buildings
buildings = pd.concat([buildings_train, buildings_test])
buildings = gpd.GeoDataFrame(buildings, geometry=gpd.points_from_xy(
    buildings.longitude, buildings.latitude, crs="EPSG:4326"
))

In [301]:
# Find all buildings missing coordinates
no_coords = buildings.latitude.isna() | buildings.longitude.isna()
street = buildings[~no_coords & (buildings.street == "пос. Коммунарка")]

buildings.loc[no_coords, "latitude"] = street.latitude.mean()
buildings.loc[no_coords, "longitude"] = street.longitude.mean()
buildings.loc[no_coords, "district"] = street.district.mode()[0]

# The coordinates of the southwest and northeast corners of a rectangle approximately encompassing Moscow
MOSCOW_SW_LAT = 55
MOSCOW_SW_LON = 35
MOSCOW_NE_LAT = 57
MOSCOW_NE_LON = 39

# Find all buildings with coordinates outside of Moscow
outside = (((buildings.latitude < MOSCOW_SW_LAT) | (buildings.latitude > MOSCOW_NE_LAT)
          | (buildings.longitude < MOSCOW_SW_LON) | (buildings.longitude > MOSCOW_NE_LON)))
buildings[outside][["split", "latitude", "longitude", "district", "street", "address", "constructed", "material", "stories"]]

#Fix coordintaes of buildings outside moscow
for idx, building in buildings[outside].iterrows():
    street = buildings[~outside & (buildings.street == building.street)]
    if len(street):
        buildings.loc[idx, "latitude"] = street.latitude.mean()
        buildings.loc[idx, "longitude"] = street.longitude.mean()
        buildings.loc[idx, "district"] = street.district.mode()[0]
    else:
        buildings.loc[idx, "latitude"] = buildings[~outside].latitude.mean()
        buildings.loc[idx, "longitude"] = buildings[~outside].longitude.mean()

In [302]:
#fix districts
no_district = buildings.district.isna()
districts = buildings.loc[no_district].apply(
    lambda b: buildings.loc[
        (buildings[~no_district][["latitude", "longitude"]] - b[["latitude", "longitude"]]).abs().sum(axis=1).idxmin()
    ].district,
    axis=1
)
districts.rename("district", inplace=True)
buildings.update(districts)

In [303]:
stations = gpd.read_file("resources/metro_stations.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# The Earth's radius in meters
EARTH_RADIUS = 6371000

# Create temporary columns for coordinates given in radians
stations["lon_rad"] = np.radians(stations.geometry.x)
stations["lat_rad"] = np.radians(stations.geometry.y)
buildings["lon_rad"] = np.radians(buildings.longitude)
buildings["lat_rad"] = np.radians(buildings.latitude)

# Calculate the distance to the nearest metro station for each building using
# the haversine formula with the Earth's radius as given above
buildings["metro_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((stations.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(stations.lat_rad)
                * np.sin((stations.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)


In [304]:
# Read park and garden location data
parks = gpd.read_file("resources/parks_and_gardens.kml", driver="KML").drop(columns=["Description"]).rename(columns={"Name": "name"})

# Create columns for coordinates given in radians
parks["lon_rad"] = np.radians(parks.geometry.x)
parks["lat_rad"] = np.radians(parks.geometry.y)

# Calculate the distance to the nearest park or garden for each building using
# the haversine formula with the Earth's radius as given above
buildings["park_distance"] = buildings.apply(
    lambda row: np.min(
        2
        * EARTH_RADIUS
        * np.arcsin(
            np.sqrt(
                np.sin((parks.lat_rad - row.lat_rad) / 2) ** 2
                + np.cos(row.lat_rad)
                * np.cos(parks.lat_rad)
                * np.sin((parks.lon_rad - row.lon_rad) / 2) ** 2
            )
        )
    ),
    axis=1,
)

# Drop the temporary radian columns
stations.drop(columns=["lon_rad", "lat_rad"], inplace=True)
buildings.drop(columns=["lon_rad", "lat_rad"], inplace=True)

In [305]:
buildings_train = buildings.loc[buildings["split"] == "train", :].copy()
buildings_train.drop(columns = "split", inplace = True)
apartments = pd.read_csv('resources/data/apartments_train.csv')
data = pd.merge(apartments, buildings_train, how='left', left_on='building_id', right_index=True)


In [306]:
distance = np.sqrt((37.6 - data["longitude"])**2 + (55.75 - data["latitude"])**2)
data["distance"] = distance

data_train, data_valid = model_selection.train_test_split(data, test_size=0.33, stratify=np.log(data.price).round())

X_train = data_train[["area_total", "longitude", "latitude", "distance", "rooms", "constructed", "district", "metro_distance", "park_distance"]]
y_train = np.log(data_train.loc[X_train.index].price)
X_valid = data_valid[["area_total", "longitude", "latitude", "distance", "rooms", "constructed", "district", "metro_distance", "park_distance"]]
y_valid = np.log(data_valid.loc[X_valid.index].price)


In [307]:
xgb_study = optuna.create_study(direction="minimize", study_name="XGB study")
func = lambda trial: objective(trial, X_train, y_train, X_valid, y_valid)
xgb_study.optimize(func, n_trials=2000)

[I 2021-10-28 13:42:47,828] A new study created in memory with name: XGB study


[13:42:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "nrounds", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:12.41288
[1]	validation-rmse:9.65666
[2]	validation-rmse:7.51243
[3]	validation-rmse:5.84569
[4]	validation-rmse:4.54916
[5]	validation-rmse:3.54081
[6]	validation-rmse:2.75774
[7]	validation-rmse:2.14904
[8]	validation-rmse:1.67661
[9]	validation-rmse:1.30952


[I 2021-10-28 13:42:47,916] Trial 0 finished with value: 1.3095221450756966 and parameters: {'nrounds': 908, 'booster': 'dart', 'lambda': 0.17790134841197075, 'alpha': 6.735483480096674e-06, 'max_depth': 9, 'eta': 0.2221307907979357, 'gamma': 0.0005138203340500229, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 4.5364582706712405e-05, 'skip_drop': 0.0004563420659903603}. Best is trial 0 with value: 1.3095221450756966.


[13:42:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "nrounds", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:18.65029
[1]	validation-rmse:24.62351
[2]	validation-rmse:31.29704
[3]	validation-rmse:34.10791
[4]	validation-rmse:38.00489
[5]	validation-rmse:29.06414
[6]	validation-rmse:33.56144
[7]	validation-rmse:43.67783
[8]	validation-rmse:54.76880
[9]	validation-rmse:71.27940


[I 2021-10-28 13:42:47,929] Trial 1 finished with value: 16.455786117452877 and parameters: {'nrounds': 691, 'booster': 'gblinear', 'lambda': 6.119230589836766e-07, 'alpha': 0.03555937545050472}. Best is trial 0 with value: 1.3095221450756966.


...

[I 2021-10-28 13:45:40,563] Trial 1997 pruned. Trial was pruned at iteration 1.


[13:45:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "nrounds", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:7.66294


[I 2021-10-28 13:45:40,696] Trial 1998 pruned. Trial was pruned at iteration 0.


[13:45:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "nrounds", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:4.12464


[I 2021-10-28 13:45:40,832] Trial 1999 pruned. Trial was pruned at iteration 0.


In [308]:
print(f"\tBest value (rmlse): {xgb_study.best_value:.5f}")
print(f"\tBest params:")

for key, value in xgb_study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmlse): 0.18183
	Best params:
		nrounds: 746
		booster: dart
		lambda: 3.983571787968778e-05
		alpha: 0.6833508661723343
		max_depth: 10
		eta: 0.9842712369409734
		gamma: 6.43682847619565e-05
		grow_policy: lossguide
		sample_type: uniform
		normalize_type: forest
		rate_drop: 0.00011371373596989981
		skip_drop: 4.393621406029479e-06


In [309]:
data_train, data_valid = model_selection.train_test_split(data, test_size=0.33, stratify=np.log(data.price).round())

X_train = data_train[["area_total", "longitude", "latitude", "distance", "rooms", "constructed", "district", "metro_distance", "park_distance", "material"]]
y_train = np.log(data_train.loc[X_train.index].price)
X_valid = data_valid[["area_total", "longitude", "latitude", "distance", "rooms", "constructed", "district", "metro_distance", "park_distance", "material"]]
y_valid = np.log(data_valid.loc[X_valid.index].price)

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

bst = xgb.train(xgb_study.best_params, dtrain, evals=[(dvalid, "validation")])



[13:45:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "nrounds" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.40951
[1]	validation-rmse:0.24364
[2]	validation-rmse:0.21390
[3]	validation-rmse:0.20641
[4]	validation-rmse:0.19699
[5]	validation-rmse:0.19487
[6]	validation-rmse:0.19123
[7]	validation-rmse:0.18848
[8]	validation-rmse:0.18652
[9]	validation-rmse:0.18561


In [310]:
dvalid_t = xgb.DMatrix(X_valid)
dtrain_t = xgb.DMatrix(X_train)

preds_t = bst.predict(dtrain_t)
preds_v = bst.predict(dvalid_t)

print(root_mean_squared_log_error(np.exp(preds_t), np.exp(y_train)))
print(mean_squared_log_error(np.exp(y_train), np.exp(preds_t),squared=False))

print("----------")

print(root_mean_squared_log_error(np.exp(preds_v), np.exp(y_valid)))
print(mean_squared_log_error(np.exp(y_valid), np.exp(preds_v),squared=False))

0.1058837555642379
0.1058837555642379
----------
0.18560972238931553
0.18560972238931553
